# Monte Carlo Methods
---

## **Tại sao cần Monte Carlo (MC)?**  

### Vấn đề với Dynamic Programming (DP):
- **Cần exact transition probabilities** - mà thực tế không có
  - Ví dụ: Dự báo thời tiết không thể cho exact probabilities
- **Computationally messy** với large state spaces
- Cần model hoàn chỉnh của environment

### MC Solution:
- Ước tính values bằng cách **average random samples**
- Giống như tung xúc xắc nhiều lần để tính trung bình
- **Model-free**: Chỉ cần experience, không cần biết transition probabilities

**💡 Core idea**: Nếu không biết exact math → dùng sampling!

## **MC Prediction: Ước tính V ≈ v_π**  

**Mục tiêu**: Tìm state value V dưới policy π

### Algorithm:

**1. Khởi tạo**:  
- `V(s)` arbitrary cho tất cả states  
- Empty list `Return(s)` cho mỗi state  

**2. Mỗi episode**:  
- **Generate episode**: S₀, A₀, R₁, S₁, ..., R_T using π  
- `G ← 0` (total discounted reward)  
- **Loop backwards** (t = T-1 to 0):  
  - Update: `G ← γG + R_{t+1}`  
  - Append G to `Return(S_t)`  
  - Update `V(S_t)` = average(`Return(S_t)`)

**🔑 Key insight**: Học từ **complete episodes**, không update giữa chừng như TD

**⚠️ Nhược điểm**: Phải chờ episode kết thúc mới update được

## **Action Values (Q-values)**  

### Tại sao cần Q thay vì chỉ V?
- **V(s)** chỉ cho biết state tốt như thế nào
- **Q(s,a)** cho biết action nào tốt nhất ở state đó!
- **Formula**: `q_π(s,a) = E[G_t | S_t=s, A_t=a]`

### Problem:
- **Cần estimates cho TẤT CẢ actions** để pick best one
- Nếu chỉ follow 1 policy → chỉ thấy 1 action per state
- **Solution**: Cần exploration!

**💭 Think about it**: Làm sao biết action nào tốt nhất nếu chưa thử hết?

## **MC Exploring Starts**  

**Mục tiêu**: Ước tính π ≈ π* (optimal policy)  

### Ý tưởng:
- **Bắt buộc** start từ mọi state-action pair có thể
- Đảm bảo tất cả (s,a) được explore

### Algorithm:
**1. Initialize**: 
- `Q(s,a)` arbitrarily + empty `Returns(s,a)`  

**2. Mỗi episode**:  
- **Exploring Start**: Random pick S₀, A₀ (tất cả pairs đều possible)  
- Generate episode using π  
- Update G backwards, append to `Returns(S_t,A_t)`  
- Set `Q(S_t,A_t)` = average(`Returns`) và `π(S_t) = argmax Q(S_t,a)`

### Vấn đề:
- ❌ **Impractical**: Không thể start từ bất kỳ state nào trong real world
- ❌ **Unrealistic assumption**: Exploring starts không có trong thực tế

**🤔 Real world**: Bạn không thể bắt đầu game chess từ bất kỳ position nào!

## **ε-Soft Policies**  

**Giải pháp cho Exploring Starts**: Đảm bảo exploration liên tục

### Định nghĩa:
- **Rule**: Mọi action có probability ≥ ε/|A(s)|
- **Nghĩa là**: Không action nào có probability = 0
- **Luôn có chance** thử action "tệ"

### Ưu điểm:
- ✅ **Không cần "start anywhere"**  
- ✅ **Luôn visit tất cả state-action pairs**  
- ✅ **Practical** trong real applications

### Nhược điểm:
- ❌ **Policy là stochastic** (không pure optimal)
- ❌ **Trade-off**: Exploration vs Exploitation

**💡 Intuition**: Thỉnh thoảng làm điều "stupid" để học được điều mới!

## **MC Control (ε-Soft Policies)**  

**Mục tiêu**: Ước tính π ≈ π* mà không cần exploring starts

### Algorithm:

**1. Initialize**:  
- Arbitrary ε-soft π, Q(s,a), empty `Returns(s,a)`  

**2. Mỗi episode**:  
- Generate episode using π  
- Update G backwards, append to `Returns(S_t,A_t)`  
- Update `Q(S_t,A_t)` = average(`Returns`)  
- **Update policy**:  
  - `A* = argmax_a Q(S_t, a)` (best action)
  - **Cho tất cả actions a ở state S_t**:
    ```
    π(a|S_t) = {
      1 - ε + ε/|A(s)|    if a = A* (best action)
      ε/|A(s)|           otherwise (other actions)
    }
    ```

### Logic:
- **Best action**: Có probability cao nhất
- **Other actions**: Vẫn có chance được chọn
- **Balance**: Exploit best + Explore others

**🎯 Result**: Policy vừa greedy vừa có exploration!

## **Off-Policy Learning**  

**Ý tưởng**: Học target policy π trong khi follow behavior policy b

### Phân biệt:
- **On-policy**: Học/evaluate **cùng** policy đang dùng để act  
- **Off-policy**: Học/evaluate **khác** policy (π) với acting policy (b)  

### Ứng dụng chính:
- **Exploration**: b = exploratory, π = optimal
- **Learning from demos**: b = human behavior, π = optimal policy
- **Safety**: b = safe policy, π = aggressive optimal policy

### Điều kiện bắt buộc - Coverage:
```
π(a|s) > 0 ⟹ b(a|s) > 0
```
**Nghĩa**: Nếu π muốn thử action → b phải thỉnh thoảng thử action đó!

**🎪 Ví dụ**: Học lái xe từ người mới học (b) để trở thành tài xế giỏi (π)

**⚠️ Critical**: Không có coverage → không học được!

## **Importance Sampling**  

**Vấn đề**: Làm sao estimate expected return của π khi chỉ có samples từ b?

### Math Trick:
```
E_π[X] = E_b[X · ρ(X)]
```
Trong đó: `ρ(X) = π(X)/b(X)` (importance sampling ratio)

### Estimate từ samples:
```
E_π[X] ≈ (1/n) ∑ᵢ xᵢ · ρ(xᵢ)
```

### Intuition:
- **ρ > 1**: π thích action này hơn b → **weight up**
- **ρ < 1**: π ít thích action này hơn b → **weight down**
- **ρ = 1**: π và b như nhau → **no adjustment**

### Vấn đề:
- **High variance**: Nếu ρ rất lớn → estimates không stable
- **Rare events**: Nếu b hiếm khi thử action mà π thích → ít data

**🎯 Bottom line**: Powerful technique nhưng cần careful với variance!

## **Ghi Chú Cá Nhân**  

### **MC vs DP**:
- **MC**: Không cần model (chỉ cần samples!), nhưng high variance  
- **DP**: Cần model đầy đủ, nhưng low variance nếu model chính xác
- **MC winner**: Khi environment unknown hoặc too complex

### **ε-soft trade-off**:
- **Small ε**: Gần như greedy, ít exploration
- **Large ε**: Nhiều exploration, nhưng policy kém optimal
- **Sweet spot**: Tùy thuộc vào problem và stage of learning

### **Off-policy pro tips**:
- Dùng b để **explore**, π để **optimize**
- **Coverage is non-negotiable** - không có thì toang!
- Importance sampling powerful nhưng **watch out for variance**

### **Khi nào dùng MC?**
- Environment unknown
- Episodes có finite length
- Cần model-free approach
- Có thời gian để collect nhiều episodes

---  